# WMT18

* To english
* Segment level data
* Relative ranking (better translation vs. worse translation)
* Kendall score

In [1]:
!pip install datasets
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 15.9 MB/s 
     |████████████████████████████████| 212 kB 65.2 MB/s 
     |████████████████████████████████| 141 kB 65.4 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 115 kB 73.8 MB/s 
     |████████████████████████████████| 596 kB 58.5 MB/s 
     |████████████████████████████████| 127 kB 61.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 8.5 MB/s 
     |██████████████████████████

In [2]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 217 (delta 6), reused 11 (delta 6), pack-reused 205
Receiving objects: 100% (217/217), 189.52 KiB | 17.23 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [3]:
import datasets
import numpy as np

from geneval.geneval.data.wmt import WMT18

## BERTScore
* RoBERTa
* F1

In [4]:
# kendall score
# https://github.com/Tiiiger/bert_score/blob/master/reproduce/get_wmt18_seg_results.py#L53

def kendall_score(scores_better, scores_worse):
    total = len(scores_better)
    correct = np.sum(np.array(scores_better) > np.array(scores_worse))
    incorrect = total - correct
    return (correct - incorrect)/total

In [5]:
bert_score = datasets.load_metric("bertscore")

In [6]:
results = {}
for lang_pair in ["cs-en", "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"]:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")
    # compute scores
    scores_better = bert_score.compute(
        predictions=wmt.translations_better,
        references=wmt.references,
        lang="en"
    )
    scores_worse = bert_score.compute(
        predictions=wmt.translations_worse,
        references=wmt.references,
        lang="en"
    )
    ks = kendall_score(scores_better["f1"], scores_worse["f1"])
    results[lang_pair] = ks

In [7]:
results

{'cs-en': 0.4140900195694716,
 'de-en': 0.5548187274292837,
 'et-en': 0.4033603074698965,
 'fi-en': 0.30610940695296524,
 'ru-en': 0.354479046520569,
 'tr-en': 0.3020527859237537,
 'zh-en': 0.2647420331564589}